<br>
<br>

# Import modules

In [1]:
#BASIC MODULES
import pandas as pd #data manipulation
import numpy as np #linear algebra
import os #operating system

#STATISTICS
import statsmodels.api as sm #logistic regression

#MODELLING AND DATA PROCESSING
from sklearn.experimental import enable_iterative_imputer #multivariate imputation
from sklearn.ensemble import RandomForestRegressor #regressor for imputation
from sklearn.impute import IterativeImputer #multivariate imputation
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

from tqdm import tqdm as tqdm
import itertools

<br>
<br>

# Open dataset

In [2]:
#open and display dataset
path = os.getcwd()
data = pd.read_csv(f"{path}/gono.csv")
data = data.drop("ID", axis=1)
print("Dataset shape:",data.shape, "\n")
data.head()

Dataset shape: (3144, 11) 



,SEXE,ETAT_C,AGE,ORIENT_SEX,MTS_ANT,NB_MTS,RAISON,NB_PART,HISTOIRE,CULTURE,DIAGN
0,1,1,29,1,2,8,3,5.0,0.0,0.0,0.0
1,1,1,18,1,1,0,2,1.0,1.0,1.0,1.0
2,1,1,24,1,2,1,1,2.0,0.0,0.0,0.0
3,1,1,29,1,2,3,1,6.0,0.0,1.0,1.0
4,1,3,28,1,2,3,1,2.0,0.0,0.0,0.0


In [3]:
#compute proportion of observations sicked
values = data["DIAGN"].value_counts()
prop = values[1] / (values[0]+values[1])
print(f"Proportion of sickness: {round(prop*100,2)}%")

Proportion of sickness: 24.94%


<br>
<br>

# Replace 9/99 by np.nan and change data types

In [4]:
# replace value 9 and 99 by NaN
liste = ["ID", "AGE", "NB_PART"]
for var in data.columns:
    if var not in liste:
        data[var] = data[var].replace(9,np.nan)
    else:
        data[var] = data[var].replace(99,np.nan)
print(f"Total numbers of NaN: {data.isna().sum().sum()}")

Total numbers of NaN: 601


In [5]:
liste = ["AGE", "NB_PART", "NB_MTS"]
for var in data.columns:
    if var not in liste:
        data[var] = data[var].astype('category')
data.dtypes

SEXE          category
ETAT_C        category
AGE            float64
ORIENT_SEX    category
MTS_ANT       category
NB_MTS         float64
RAISON        category
NB_PART        float64
HISTOIRE      category
CULTURE       category
DIAGN         category
dtype: object

<br>
<br>

# Imputation NA (long execution time)

In [6]:
print(f"Total numbers of nan (before imputation): {data.isna().sum().sum()}")

#define a multivariate imputer 
imp = IterativeImputer(estimator=RandomForestRegressor(), max_iter=20)

#imputation 
data_to_imput = data.copy()
imp.fit(data_to_imput)
data_imputed = pd.DataFrame(imp.transform(data_to_imput), columns = data_to_imput.columns)
data_imputed = round(data_imputed) #round because the imputer uses a regressor

#display new dataset
print(f"\nTotal numbers of nan (after imputation): {data_imputed.isna().sum().sum()}")
data_imputed.head()

Total numbers of nan (before imputation): 601


/Users/josephbarbier/opt/anaconda3/lib/python3.9/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(



Total numbers of nan (after imputation): 0


,SEXE,ETAT_C,AGE,ORIENT_SEX,MTS_ANT,NB_MTS,RAISON,NB_PART,HISTOIRE,CULTURE,DIAGN
0,1.0,1.0,29.0,1.0,2.0,8.0,3.0,5.0,0.0,0.0,0.0
1,1.0,1.0,18.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0
2,1.0,1.0,24.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0
3,1.0,1.0,29.0,1.0,2.0,3.0,1.0,6.0,0.0,1.0,1.0
4,1.0,3.0,28.0,1.0,2.0,3.0,1.0,2.0,0.0,0.0,0.0


<br>
<br>

# Split

In [7]:
data_imputed.head()

,SEXE,ETAT_C,AGE,ORIENT_SEX,MTS_ANT,NB_MTS,RAISON,NB_PART,HISTOIRE,CULTURE,DIAGN
0,1.0,1.0,29.0,1.0,2.0,8.0,3.0,5.0,0.0,0.0,0.0
1,1.0,1.0,18.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0
2,1.0,1.0,24.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0
3,1.0,1.0,29.0,1.0,2.0,3.0,1.0,6.0,0.0,1.0,1.0
4,1.0,3.0,28.0,1.0,2.0,3.0,1.0,2.0,0.0,0.0,0.0


In [8]:
#split the dataset into a train and a test set
X = data_imputed.drop(["DIAGN", "CULTURE"], axis=1)
X = X.assign(Intercept=1)
y = data_imputed["DIAGN"]
print(X.shape)
print(y.shape)

(3144, 10)
(3144,)


<br>

# Backward elimination

In [9]:
model = sm.GLM(y, X, family=sm.families.Binomial())
results = model.fit()

#stepwise algorithm
while results.pvalues.max() > 0.15:
    max_pvalue_index = results.pvalues.idxmax()
    X = X.drop(max_pvalue_index, axis=1)
    logit_model = sm.Logit(y, X)
    results = logit_model.fit()

#display "best" model results    
print("Model with most significant variables\n")
results.summary()

Optimization terminated successfully.
         Current function value: 0.527750
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.527894
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.528077
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.528311
         Iterations 6
Model with most significant variables



<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  DIAGN   No. Observations:                 3144
Model:                          Logit   Df Residuals:                     3138
Method:                           MLE   Df Model:                            5
Date:                Wed, 15 Mar 2023   Pseudo R-squ.:                 0.04850
Time:                        11:39:09   Log-Likelihood:                -1661.0
converged:                       True   LL-Null:                       -1745.7
Covariance Type:            nonrobust   LLR p-value:                 1.019e-34
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
SEXE          -0.4188      0.143     -2.920      0.003      -0.700      -0.138
AGE           -0.0224      0.006     -3.787      0.000      -0.034      -0.011
ORIENT_SEX    -0.7638      0.109     -7.019      0.000      -0.977      -0.551
NB_PART        0.0302      0.007      4.257      0.000       0.016       0.044
HISTOIRE       0.2363      0.111      2.128      0.033       0.019       0.454
Intercept      0.9103      0.246      3.699      0.000       0.428       1.393
==============================================================================
"""

In [10]:
X = data_imputed[["SEXE", "AGE", "NB_PART", "HISTOIRE"]]
var_to_dummies = ["SEXE", "HISTOIRE"]

#add the new variable-matrix and remove the older ones
for var in var_to_dummies:
    to_add = pd.get_dummies(X[var], drop_first=True, prefix=var)
    X = X.drop(var, axis=1)
    X = pd.concat([X, to_add], axis=1)
X = X.assign(Intercept=1)
X.head()

,AGE,NB_PART,SEXE_2.0,HISTOIRE_1.0,Intercept
0,29.0,5.0,0,0,1
1,18.0,1.0,0,1,1
2,24.0,2.0,0,0,1
3,29.0,6.0,0,0,1
4,28.0,2.0,0,0,1


<br>
<br>

# Cross validation

In [11]:
models_aic = []
models_bic = []
n_folds = 5
logreg = LogisticRegression()


for i in tqdm(range(1, len(X.columns) + 1)):
    for combo in itertools.combinations(X.columns, i):
        X_subset = X[list(combo)]
        # Effectuer la validation croisée
        kf = KFold(n_splits=n_folds, shuffle=True)
        aic_scores = []
        bic_scores = []
        for train_index, test_index in kf.split(X_subset):
            X_train, X_test = X_subset.iloc[train_index], X_subset.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            logreg.fit(X_train, y_train)
            aic_scores.append(logreg.score(X_test, y_test) - 2 * i)
            bic_scores.append(logreg.score(X_test, y_test) - i * np.log(len(y_test)))
        # Stocker les résultats
        models_aic.append((list(combo), np.mean(aic_scores)))
        models_bic.append((list(combo), np.mean(bic_scores)))
best_aic = sorted(models_aic, key=lambda x: x[1])[0]
best_bic = sorted(models_bic, key=lambda x: x[1])[0]
print(f"Best AIC model: {best_aic[0]} with AIC score of {best_aic[1]}")
print(f"Best BIC model: {best_bic[0]} with BIC score of {best_bic[1]}")

100%|██████████| 5/5 [00:02<00:00,  1.85it/s]

Best AIC model: ['AGE', 'NB_PART', 'SEXE_2.0', 'HISTOIRE_1.0', 'Intercept'] with AIC score of -9.24205441859994
Best BIC model: ['AGE', 'NB_PART', 'SEXE_2.0', 'HISTOIRE_1.0', 'Intercept'] with BIC score of -31.461119611869908
